### Import **HR Analysis case study** dataset

In [94]:
!pip install -q kagglehub pandas scikit-learn

In [95]:
import kagglehub
from pathlib import Path

# Download latest version
path = Path(kagglehub.dataset_download("vjchoudhary7/hr-analytics-case-study", path="general_data.csv"))

Path('hr_data_raw.csv').write_text(path.read_text());

### Clear data

In [96]:
import pandas as pd

df = pd.read_csv("hr_data_raw.csv", delimiter=',', decimal='.')
# drop unused columns
df = df.drop(columns=["EmployeeID", "EmployeeCount", "Over18", "StandardHours"])
print("Null values:")
display(df.isna().sum()[df.isna().sum() > 0])
# drop null values
df = df.dropna(how='any')

Null values:


NumCompaniesWorked    19
TotalWorkingYears      9
dtype: int64

### Save silver

In [97]:
df.to_csv("hr_data_silver.csv", index=False)

### Encode


In [98]:
# binary encoding
columns = ["Attrition"]

for c in columns:
    values = sorted(df[c].unique())
    df[c] = df[c].map({values[0]: 0, values[1]: 1})

In [99]:
# one hot encoding
columns = ["BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus"]
for column in columns:
    dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(columns=column)
df = df.replace({True: 1, False: 0})

/tmp/ipykernel_69650/2721956685.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [100]:
display(list(df.columns))

['Age',
 'Attrition',
 'DistanceFromHome',
 'Education',
 'JobLevel',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'YearsAtCompany',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager',
 'BusinessTravel_Travel_Frequently',
 'BusinessTravel_Travel_Rarely',
 'Department_Research & Development',
 'Department_Sales',
 'EducationField_Life Sciences',
 'EducationField_Marketing',
 'EducationField_Medical',
 'EducationField_Other',
 'EducationField_Technical Degree',
 'Gender_Male',
 'JobRole_Human Resources',
 'JobRole_Laboratory Technician',
 'JobRole_Manager',
 'JobRole_Manufacturing Director',
 'JobRole_Research Director',
 'JobRole_Research Scientist',
 'JobRole_Sales Executive',
 'JobRole_Sales Representative',
 'MaritalStatus_Married',
 'MaritalStatus_Single']

### Normalize numeric values

In [101]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

columns = ["Age", "DistanceFromHome", "Education", "JobLevel", "MonthlyIncome", "NumCompaniesWorked", "PercentSalaryHike", "StockOptionLevel", "TotalWorkingYears", "TrainingTimesLastYear", "YearsAtCompany", "YearsSinceLastPromotion", "YearsWithCurrManager"]

scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()

scaled_columns = scaler_minmax.fit_transform(df[columns])
df_normalized = scaler_standard.fit_transform(scaled_columns)

df[columns] = df_normalized

### Equalize numer of males and females

In [102]:
print(df["Gender_Male"].map({1: "Male", 0: "Female"}).value_counts())

Gender_Male
Male      2626
Female    1756
Name: count, dtype: int64


In [103]:
# Oversample SMOTE Females
!pip install -q imbalanced-learn

from imblearn.over_sampling import SMOTE

features = df.drop(columns=["Gender_Male"])
target = df["Gender_Male"]

smote = SMOTE(sampling_strategy='auto', random_state=42)

features_rs, target_rs = smote.fit_resample(features, target)

df = pd.DataFrame(features_rs, columns=features.columns)
df['Gender_Male'] = target_rs


In [ ]:
df['Gender_Male'].value_counts()

### Save Gold ds

In [104]:
df.to_csv("hr_data_gold.csv", index=False)